В этом ноутбуке кортежи мнений предсказываются с помощью модели Mistral Large 2 (mistral-large-latest).

Для использования необходим Mistral API key, который можно получить бесплатно.

## Проверка ##

In [20]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

api_key = "YOUR_MISTRAL_API_KEY"
model = "mistral-large-latest"
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
SEED = 42

random.seed(SEED)

In [21]:
response = requests.get(url)
response.headers

{'Date': 'Wed, 08 Oct 2025 13:45:21 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '25', 'Connection': 'keep-alive', 'CF-RAY': '98b614037f5df12b-DME', 'mistral-correlation-id': '0199c411-be5c-730c-b995-d15ac5aea094', 'x-kong-request-id': '0199c411-be5c-730c-b995-d15ac5aea094', 'access-control-allow-origin': '*', 'x-kong-response-latency': '0', 'cf-cache-status': 'DYNAMIC', 'Set-Cookie': '__cf_bm=Lvv5MrOH3IfRT.2LdTxzpzX1C9_fUwmnGbYNoyjSfDA-1759931121-1.0.1.1-3gtwETPyx7MLc5DUQxLbxP.gaXVKHXE3.Oyd6ft86MEIJbd0YVCtwOyV0GYgJDLX4y7lB0fbqPUwARf1IBIwsjmKiGSZUQr9hKED_wEmi8o; path=/; expires=Wed, 08-Oct-25 14:15:21 GMT; domain=.mistral.ai; HttpOnly; Secure; SameSite=None, _cfuvid=GhGvXKthYAaiIclkRH6ziIio3Pni2uSh7rHKQn.KR4k-1759931121258-0.0.1.1-604800000; path=/; domain=.mistral.ai; HttpOnly; Secure; SameSite=None', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains', 'Server': 'cloudflare', 'alt-svc': 'h3=":443"; ma=86400'}

In [14]:
class MistralModel():
    def __init__(self, model, temp = 0.2, top_p = 0.9, max_tokens = 256, seed = 42):
        self.model = model
        self.temp = temp
        self.top_p = top_p
        self.max_tokens = max_tokens
        self.seed = seed
    def prompt(self, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'user', 'content': text}]
             }
        return prompt
    
    def structured_prompt(self, instruction, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'system', 'content': instruction},
                          {'role':'user', 'content': text}]
             }
        return prompt

Mistral = MistralModel(model, max_tokens = 512)

instruction = "ответь на вопрос"
text = "кто тебя создал?"
response = requests.post(url, headers=headers, json=Mistral.prompt(text))
print(f'Вопрос: {text}')
print(f"Ответ: {response.json()['choices'][0]['message']['content']}")

Вопрос: кто тебя создал?
Ответ: Меня создал Mistral AI, передовой стартап в области искусственного интеллекта из Франции.


## Подготовка данных ##

In [15]:
import requests, zipfile, io

# download utility functions
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

from src.utils import parse_data, load_jsonl, to_jsonl, form_prompt, dict2tuple, extract_tuple, str2list, save_jsonl, df2structure

In [16]:
# download train and test data
train_path = "train.jsonl"
test_path = "test.jsonl"

url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/train.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/test.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 803


In [17]:
# download distances file
dists = requests.get('https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/test_distances.txt')
dists = [x.split() for x in dists.text.strip().split('\n')]

print(dists[0][:5])
print(dists[-1][:5])

['307', '1254', '22', '296', '2517']
['1459', '1586', '210', '504', '1438']


In [10]:
# csv's with preds, generated in Stage1

preds1 = pd.read_csv('PREDS1.csv')
preds2 = pd.read_csv('PREDS2.csv')
preds3 = pd.read_csv('PREDS3.csv')

aug = [preds1, preds2, preds3]

## Определения ##

In [19]:
def aug_prompt(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
{shots}
Текст, который нужно проанализировать:
{text}
Ответы экспертов к этому тексту:
{variants}
Ты можешь выбрать из этих ответов или ответить по-своему.
Твой ответ:"""

def aug_prompt_eng(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""You are an expert in sentiment analysis.
You need to identify all positive and negative relations between entities in the text and present them in the following format:
[source of sentiment, target, polar expression, polarity (POS/NEG)]
If the author is the source of sentiment, write:
['AUTHOR', target, polar expression, polarity (POS/NEG)]
If there is no explicit source, write:
['NULL', target, polar expression, polarity (POS/NEG)]
Returning an empty answer is allowed:
[]
Do not provide any explanations in the response.
Examples
{shots}
Text to analyze: {text}
Expert answers to this text:
{variants}
You can choose from these answers or provide your own answer.
Your answer:"""

class Runner():
    def __init__(self, model, url, headers, train, test, priorities, aug, n_shots = 5, sleeptime = 2):
        self.model = model
        self.url = url
        self.headers = headers
        self.train = train
        self.test = test
        self.priorities = priorities
        self.aug = aug
        self.n_shots = n_shots
        self.sleeptime = sleeptime
    
    def run(self):
        results = list()
        for i in tqdm(range(len(self.test))):
            time.sleep(self.sleeptime)
            entry = self.test[i]
            # for random choice
            # examples = [dict2tuple(x) for x in choices(self.train, k = self.n_shots)]
            examples = [self.train[int(j)] for j in self.priorities[i][:self.n_shots]]
            examples = [dict2tuple(x) for x in examples]
            variants = [x.loc[i]['pred'] for x in self.aug]
            # English prompt
            # prompt = aug_prompt_eng(examples, variants, entry['text'])
            prompt = aug_prompt(examples, variants, entry['text'])
            response = requests.post(self.url,
                                     headers=self.headers,
                                     json=self.model.prompt(prompt))
            result = []
            if response.status_code == 200:
                response = response.json()['choices'][0]['message']['content']
                try:
                    result = ast.literal_eval(extract_tuple(response))
                except (SyntaxError, ValueError):
                    print(f'bad response, iteration:{len(results)}')
            else:
                print(f'bad response, iteration:{len(results)}') 
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

## Инференс ##

In [14]:
%%time
params = {'temp': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
Mistral = MistralModel(model, **params)
n_shots = 12
runner = Runner(Mistral, url, headers, train, test, dists, aug, n_shots, sleeptime = 3)

path = "YOUR PATH"
print(len(runner.priorities), len(runner.aug))
result = runner.run()

803 3


100%|██████████| 803/803 [59:20<00:00,  4.43s/it]  

CPU times: total: 3min 1s
Wall time: 59min 20s


In [15]:
path = "YOUR_PATH"
print(path)
print(result[13])

./results/Mistral/Mistral_claug_12shot_0.1temp_eng
(13, 'В Пекине в настоящий момент зарегистрировано 6 кольцевых дорог и 4.76 миллиона транспортных средств, по загруженности китайская столица занимает сомнительное первое место в мире вместе с мексиканском Мехико.', [], [['NULL', 'Пекин', 'сомнительное первое место в мире', 'NEG'], ['NULL', 'Мехико', 'сомнительное первое место в мире', 'NEG']])


## Результаты ##

сохраняем csv

In [16]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],"[[NULL, Борис Моисеев, перенес инсульт, NEG]]"
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]
...,...,...,...,...
798,798,"Уилер, выпускник военной Академии в Вест-Пойнт...",[],[]
799,799,"В Америке известен тем, что, будучи председате...",[],"[[NULL, председатель фонда Мемориала войны во ..."
800,800,66-летний Уилер жил вместе с женой в Ньюкасле ...,[],[]
801,801,"Известно, что 28 декабря он должен был приехат...",[],[]


In [17]:
print(path)
save(output, path)

./results/Mistral/Mistral_claug_12shot_0.1temp_eng


In [18]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],"[[NULL, Борис Моисеев, перенес инсульт, NEG]]"
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]
...,...,...,...,...
798,798,"Уилер, выпускник военной Академии в Вест-Пойнт...",[],[]
799,799,"В Америке известен тем, что, будучи председате...",[],"[[NULL, председатель фонда Мемориала войны во ..."
800,800,66-летний Уилер жил вместе с женой в Ньюкасле ...,[],[]
801,801,"Известно, что 28 декабря он должен был приехат...",[],[]


In [19]:
save(output, path, raw = False)

## csv to jsonl ##

переводим в формат для подачи на RuOpinionNE-2024

In [20]:
final = df2structure(output)
final[13]

{'sent_id': 13,
 'text': 'В Пекине в настоящий момент зарегистрировано 6 кольцевых дорог и 4.76 миллиона транспортных средств, по загруженности китайская столица занимает сомнительное первое место в мире вместе с мексиканском Мехико.',
 'opinions': [{'Source': [['NULL'], ['0:0']],
   'Target': [['Пекин'], ['2:7']],
   'Polar_expression': [['сомнительное первое место в мире'], ['145:177']],
   'Polarity': 'NEG'},
  {'Source': [['NULL'], ['0:0']],
   'Target': [['Мехико'], ['200:206']],
   'Polar_expression': [['сомнительное первое место в мире'], ['145:177']],
   'Polarity': 'NEG'}]}

In [ ]:
# this may cause UnicodeEncodeError when executed locally
save_jsonl(final, path)